Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
start = time.time()
very_start = time.time()

In [2]:
#import pandas as pd, 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import cudf, cupy, time, rmm
cudf.__version__

'0.14.0'

In [3]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [4]:
cluster = LocalCUDACluster(ip='10.2.61.36')#,protocol="ucx", 
                           #rmm_pool_size="26GB",
                           #enable_tcp_over_ucx=True, enable_nvlink=True)
#cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://10.2.61.36:43723 Dashboard: http://10.2.61.36:8787/status,Cluster Workers: 4 Cores: 4 Memory: 270.39 GB


In [5]:
%%time
path = '/raid/data/recsys/dask_input/step1_output'
df = dask_cudf.read_parquet(f'{path}/*.parquet')
df, = dask.persist(df)
_ = wait(df)
df.head()

CPU times: user 1.06 s, sys: 352 ms, total: 1.41 s
Wall time: 3.48 s


,hashtags,tweet_id,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,...,reply,retweet,retweet_comment,like,id,language,tweet_type,media,a_user_id,b_user_id
0,None,82106226,None,None,1581016405,429,605,False,1475647574,65,...,0,1581018257,0,1581018257,18689,63,1,12,29278002,17389071
1,None,725094422,None,None,1581115466,19125,142,False,1368466630,92,...,0,0,0,0,18690,59,2,12,21253110,17260406
2,None,187456866,None,None,1581369156,554,205,False,1520878712,94,...,0,1581399261,0,1581399261,18691,59,1,12,18400787,17260406
3,None,376529151,None,None,1581483924,102476,90,False,1507431791,21,...,0,0,0,0,18692,54,2,4,30346215,99628
4,99AB4BF985AFE868CA7C8458146C5271,356861582,None,None,1581391904,1912096,179,True,1245772264,21,...,0,0,0,1581414157,18693,54,1,12,22360189,99628


In [6]:
%%time
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token)+1)*not_null).fillna(0)
df['len_hashtags'] = df['hashtags'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_domains']  = df['domains'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_links']    = df['links'].map_partitions(lambda ds: count_token(ds,'\t'))

df, = dask.persist(df)
_ = wait(df)
df.head()

CPU times: user 299 ms, sys: 14.2 ms, total: 314 ms
Wall time: 568 ms


,hashtags,tweet_id,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,...,like,id,language,tweet_type,media,a_user_id,b_user_id,len_hashtags,len_domains,len_links
0,None,82106226,None,None,1581016405,429,605,False,1475647574,65,...,1581018257,18689,63,1,12,29278002,17389071,0,0,0
1,None,725094422,None,None,1581115466,19125,142,False,1368466630,92,...,0,18690,59,2,12,21253110,17260406,0,0,0
2,None,187456866,None,None,1581369156,554,205,False,1520878712,94,...,1581399261,18691,59,1,12,18400787,17260406,0,0,0
3,None,376529151,None,None,1581483924,102476,90,False,1507431791,21,...,0,18692,54,2,4,30346215,99628,0,0,0
4,99AB4BF985AFE868CA7C8458146C5271,356861582,None,None,1581391904,1912096,179,True,1245772264,21,...,1581414157,18693,54,1,12,22360189,99628,1,0,0


In [7]:
def most_frequent_token(df, col, idcol, outcol, sep='\t'):
    ds = df[[idcol,col]].compute()
    tokens = ds[col].str.tokenize(sep)
    
    token_counts = ds[col].str.token_count('\t').to_frame()
    token_counts[idcol] = ds[idcol]
    del ds
    token_rows = reconstruct_token_rows(token_counts,col,idcol)
    del token_counts
    
    tokens = tokens.to_frame()
    tokens.columns = ['token']
    tokens['token'],_ = tokens['token'].factorize()
    tokens['token'] = tokens['token']+1
    tokens[idcol] = token_rows
    
    global_token_counts = tokens['token'].value_counts().reset_index()
    global_token_counts.columns = ['token','count']
    
    tokens = tokens.merge(global_token_counts,on='token',how='left')
    del global_token_counts
    
    token_counts = tokens.groupby(idcol).agg({'count':'max'}).reset_index()
    tokens = tokens.merge(token_counts,on=idcol,how='left')
    del token_counts
    mask = tokens.count_x == tokens.count_y
    tokens = tokens[mask]
    tokens = tokens.drop(['count_x','count_y'],axis=1)
    tokens = tokens.drop_duplicates(subset=['token',idcol])
    tokens = tokens.rename(mapper={'token':outcol})
    
    if outcol == col:
        df = df.drop(col,axis=1)
    df = df.merge(tokens,on=idcol,how='left')
    df[outcol] = df[outcol].fillna(0)
    del tokens
    return df

def reconstruct_token_rows(token_counts,col,idcol):
    ids = cupy.asnumpy(token_counts[idcol].values)
    rep = cupy.asnumpy(token_counts[col].values)
    token_rows = np.repeat(ids,rep)
    token_rows = cupy.asarray(token_rows)
    return token_rows


In [8]:
%%time
for col in ['domains','links','hashtags']:
    df = most_frequent_token(df, col=col, idcol='id', outcol=col, sep='\t')
    df, = dask.persist(df)
    _ = wait(df)
df.head()

CPU times: user 14.1 s, sys: 16.1 s, total: 30.2 s
Wall time: 43 s


,tweet_id,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,...,tweet_type,media,a_user_id,b_user_id,len_hashtags,len_domains,len_links,domains,links,hashtags
0,836939972,1580970274,4879,4887,False,1409422288,14675,15049,False,1336421092,...,2,4,18249810,17151597,1,0,0,0,0,1641851
1,246147565,1581006667,5601,513,False,1236832364,2532,758,False,1237123999,...,2,12,18235619,17275531,11,1,1,383668,4411742,1022679
2,17949907,1581530401,10339941,275,True,1175467395,18,197,False,1557021150,...,2,8,17593575,73038,2,1,1,86003,5099217,1676966
3,419763933,1581455765,7183600,1334,True,1221538577,456,635,False,1319158411,...,1,12,29825412,150220,2,0,0,0,0,316584
4,69281605,1581544301,17484,16983,False,1455267394,22,82,False,1498132616,...,1,8,21671318,71905,3,0,0,0,0,1587087


In [9]:
%%time

TRAIN_SIZE = 121386431
VALID_SIZE = 12434735

train = df[df['id']<=TRAIN_SIZE] # df['id'].min() = 1
test0 = df[(df['id']>TRAIN_SIZE)&(df['id']<=TRAIN_SIZE+VALID_SIZE)]
test1 = df[df['id']>TRAIN_SIZE+VALID_SIZE]

print(len(train),len(test0),len(test1))

121868549 12490390 12490585
CPU times: user 1.1 s, sys: 16.2 ms, total: 1.11 s
Wall time: 1.46 s


In [10]:
%%time
path = '/raid/data/recsys/dask_input/step2_output'

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [11]:
%%time
train.to_parquet(f'{path}/train.parquet', write_index=False)
test0.to_parquet(f'{path}/valid.parquet', write_index=False)
test1.to_parquet(f'{path}/test.parquet', write_index=False)

CPU times: user 452 ms, sys: 77.9 ms, total: 530 ms
Wall time: 4.54 s


In [12]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 1.0 minutes
